In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import log_loss
%matplotlib inline
print("Seaborn version: ", sns.__version__)

Seaborn version:  0.9.0


In [2]:
teams = pd.read_csv('D1_teams.csv')
teams.drop(columns=['code_ncaa','school_ncaa','turner_name','league_name','league_alias','conf_alias',
                    'conf_id','division_name','division_alias','division_id','venue_id'], inplace=True)
teams.info()
teams.set_index('id',inplace=True)
teams.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 3 columns):
id                351 non-null object
kaggle_team_id    351 non-null int64
conf_name         351 non-null object
dtypes: int64(1), object(2)
memory usage: 8.3+ KB


,kaggle_team_id,conf_name
id,,
fe406882-9f22-495e-9df6-ef357a6803c6,1343,Ivy
ca478771-aa3d-4231-81e0-b70f519134fb,1463,Ivy
5c7bf63f-bc39-43c5-9907-73b50b7a6b34,1217,Ivy
d60357bd-1205-42e9-9092-d986a2843a34,1171,Ivy
88ff8c00-958e-4ccf-a21d-77fab9e93692,1165,Ivy


In [3]:
summary_2015_data = pd.read_csv('Season_2015_summary.csv')
summary_2015_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 18 columns):
market                351 non-null object
team_id               351 non-null object
points_avg            351 non-null float64
opp_pts_avg           351 non-null float64
possesion_avg         351 non-null float64
fg_pct                351 non-null float64
allow_fg_pct          351 non-null float64
off_rebs_avg          351 non-null float64
allow_off_rebs_avg    351 non-null float64
def_rebs_avg          351 non-null float64
allow_def_rebs_avg    351 non-null float64
ft_att_avg            351 non-null float64
allow_ft_att_avg      351 non-null float64
ft_pct                351 non-null float64
turnover_avg          351 non-null float64
take_away_avg         351 non-null float64
win_pct               351 non-null float64
off_rating            351 non-null float64
dtypes: float64(16), object(2)
memory usage: 49.4+ KB


In [4]:
game_data = pd.read_csv('tournament_results.csv')
game_data.drop(columns=['days_from_epoch'],inplace=True)
game_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2117 entries, 0 to 2116
Data columns (total 26 columns):
season                 2117 non-null int64
round                  2117 non-null int64
game_date              2117 non-null object
day                    2117 non-null object
win_seed               2117 non-null int64
win_region             2117 non-null object
win_market             2117 non-null object
win_name               2117 non-null object
win_alias              2117 non-null object
win_team_id            2117 non-null object
win_school_ncaa        2117 non-null object
win_code_ncaa          2117 non-null int64
win_kaggle_team_id     2117 non-null int64
win_pts                2117 non-null int64
lose_seed              2117 non-null int64
lose_region            2117 non-null object
lose_market            2117 non-null object
lose_name              2117 non-null object
lose_alias             2117 non-null object
lose_team_id           2117 non-null object
lose_school_ncaa    

In [77]:
tourn_data_2016 = game_data[game_data['season']==2016].copy()
tourn_data_2016.drop(columns=['day','win_region','win_alias','lose_region','lose_alias','lose_code_ncaa',
                              'win_school_ncaa','win_code_ncaa','num_ot','academic_year'],inplace=True)
tourn_data_2016.shape

(67, 16)

In [12]:
tourn_data_2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67 entries, 36 to 2028
Data columns (total 16 columns):
season                 67 non-null int64
round                  67 non-null int64
game_date              67 non-null object
win_seed               67 non-null int64
win_market             67 non-null object
win_name               67 non-null object
win_team_id            67 non-null object
win_kaggle_team_id     67 non-null int64
win_pts                67 non-null int64
lose_seed              67 non-null int64
lose_market            67 non-null object
lose_name              67 non-null object
lose_team_id           67 non-null object
lose_school_ncaa       67 non-null object
lose_kaggle_team_id    67 non-null int64
lose_pts               67 non-null int64
dtypes: int64(8), object(8)
memory usage: 8.9+ KB


In [78]:
tourn_data_2016.reset_index(drop=True,inplace=True)
tourn_data_2016.head()

,season,round,game_date,win_seed,win_market,win_name,win_team_id,win_kaggle_team_id,win_pts,lose_seed,lose_market,lose_name,lose_team_id,lose_school_ncaa,lose_kaggle_team_id,lose_pts
0,2016,64,2016-03-18,10,Virginia Commonwealth,Rams,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,1433,75,7,Oregon State,Beavers,532d3874-b4b3-4c5c-acc6-749a6db26c8f,Oregon St.,1333,67
1,2016,64,2016-03-17,3,Utah,Utes,0d037a5d-827a-44dd-8b70-57603d671d5d,1428,80,14,Fresno State,Bulldogs,dd8db4d8-d984-4cab-b7f6-22c6b8c2c45f,Fresno St.,1201,69
2,2016,64,2016-03-18,1,Oregon,Ducks,1da70895-f77f-44ef-b216-d63c02e696eb,1332,91,16,Holy Cross,Crusaders,31aedd91-a77e-46c1-8bdc-80e9860c159d,Holy Cross,1221,52
3,2016,16,2016-03-24,1,Oregon,Ducks,1da70895-f77f-44ef-b216-d63c02e696eb,1332,82,4,Duke,Blue Devils,faeb1160-5d15-4f26-99fc-c441cf21fc7f,Duke,1181,68
4,2016,32,2016-03-20,1,Oregon,Ducks,1da70895-f77f-44ef-b216-d63c02e696eb,1332,69,8,Saint Joseph's (PA),Hawks,f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8,Saint Joseph's,1386,64


In [203]:
columns =['season','round','game_date','game_result',
          'team','team_seed', 'team_id',
          'opp_team','opp_team_seed','opp_team_id',
          'team_pts_avg','team_allow_pts_avg',
          'team_fg_pct','team_allow_fg_pct',
          'team_off_rebs_avg','team_allow_off_rebs_avg',
          'team_def_rebs_avg','team_allow_def_rebs_avg',
          'team_ft_att_avg','team_allow_ft_att_avg',
          'team_ft_pct',
          'team_turnover_avg','team_take_away_avg',
          'team_win_pct','team_off_rating',
          'opp_pts_avg','opp_allow_pts_avg',
          'opp_fg_pct','opp_allow_fg_pct',
          'opp_off_rebs_avg','opp_allow_off_rebs_avg',
          'opp_def_rebs_avg','opp_allow_def_rebs_avg',
          'opp_ft_att_avg','opp_allow_ft_att_avg',
          'opp_ft_pct',
          'opp_turnover_avg','opp_take_away_avg',
          'opp_win_pct','opp_off_rating'  
         ]
tourn_games = pd.DataFrame(columns=columns)
tourn_games

,season,round,game_date,game_result,team,team_seed,team_id,opp_team,opp_team_seed,opp_team_id,...,opp_allow_off_rebs_avg,opp_def_rebs_avg,opp_allow_def_rebs_avg,opp_ft_att_avg,opp_allow_ft_att_avg,opp_ft_pct,opp_turnover_avg,opp_take_away_avg,opp_win_pct,opp_off_rating


In [211]:
tourn_games = pd.DataFrame(columns=columns,index=range( tourn_data_2016.shape[0]),dtype='int32')
#tourn_games[['season','round','game_date','game_result','team','team_seed',
#             'team_id']] = even_winners[['season','round','game_date','game_result','team',
                                         'team_seed','team_id']]

#df = pd.concat([tourn_games, even_winners[['season','round','game_date','game_result','team_seed',
#                                          'team','team_id']]], axis=1)

#tourn_games.game_result = tourn_games.game_result.astype(int) 
tourn_games

,season,round,game_date,game_result,team,team_seed,team_id,opp_team,opp_team_seed,opp_team_id,...,opp_allow_off_rebs_avg,opp_def_rebs_avg,opp_allow_def_rebs_avg,opp_ft_att_avg,opp_allow_ft_att_avg,opp_ft_pct,opp_turnover_avg,opp_take_away_avg,opp_win_pct,opp_off_rating
0,2016.0,64.0,2016-03-18,1.0,Virginia Commonwealth,10.0,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016.0,64.0,2016-03-18,1.0,Oregon,1.0,1da70895-f77f-44ef-b216-d63c02e696eb,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016.0,32.0,2016-03-20,1.0,Oregon,1.0,1da70895-f77f-44ef-b216-d63c02e696eb,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016.0,32.0,2016-03-20,1.0,Texas A&M,3.0,ef184799-d79b-49d9-a662-991eaf4044cd,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016.0,68.0,2016-03-15,1.0,Florida Gulf Coast,16.0,5326c079-3dc3-468f-a601-b776dd9c1e62,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2016.0,32.0,2016-03-20,1.0,Syracuse,10.0,8cd24de1-2be8-4b77-bcfa-47be0495a5f4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2016.0,64.0,2016-03-18,1.0,Syracuse,10.0,8cd24de1-2be8-4b77-bcfa-47be0495a5f4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2016.0,64.0,2016-03-18,1.0,Wisconsin,7.0,c7569eae-5b93-4197-b204-6f3a62146b25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2016.0,64.0,2016-03-17,1.0,Connecticut,9.0,efba5350-79bc-47aa-a189-db177b95b491,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2016.0,8.0,2016-03-26,1.0,Oklahoma,2.0,b2fda957-e15c-4fb2-8a13-6e58496f561e,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [218]:
games = tourn_data_2016.drop(columns=['win_pts','win_kaggle_team_id','lose_school_ncaa','lose_kaggle_team_id','lose_pts'])
games['game_result'] = 1
games.game_result = games.game_result.astype(int)
games.rename(columns={"win_seed":"team_seed","win_market":"team","win_team_id":"team_id"}, inplace=True)
games.rename(columns={"lose_seed":"opp_team_seed","lose_market":"opp_team","lose_team_id":"opp_team_id"}, inplace=True)
games


,season,round,game_date,team_seed,team,win_name,team_id,opp_team_seed,opp_team,lose_name,opp_team_id,game_result
0,2016,64,2016-03-18,10,Virginia Commonwealth,Rams,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,7,Oregon State,Beavers,532d3874-b4b3-4c5c-acc6-749a6db26c8f,1
1,2016,64,2016-03-17,3,Utah,Utes,0d037a5d-827a-44dd-8b70-57603d671d5d,14,Fresno State,Bulldogs,dd8db4d8-d984-4cab-b7f6-22c6b8c2c45f,1
2,2016,64,2016-03-18,1,Oregon,Ducks,1da70895-f77f-44ef-b216-d63c02e696eb,16,Holy Cross,Crusaders,31aedd91-a77e-46c1-8bdc-80e9860c159d,1
3,2016,16,2016-03-24,1,Oregon,Ducks,1da70895-f77f-44ef-b216-d63c02e696eb,4,Duke,Blue Devils,faeb1160-5d15-4f26-99fc-c441cf21fc7f,1
4,2016,32,2016-03-20,1,Oregon,Ducks,1da70895-f77f-44ef-b216-d63c02e696eb,8,Saint Joseph's (PA),Hawks,f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8,1
5,2016,64,2016-03-18,8,Saint Joseph's (PA),Hawks,f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8,9,Cincinnati,Bearcats,a17dfc54-415f-49c3-a2e6-d6fa9db18b0a,1
6,2016,32,2016-03-20,3,Texas A&M,Aggies,ef184799-d79b-49d9-a662-991eaf4044cd,11,Northern Iowa,Panthers,2c93a75d-5449-4ec1-9ca2-1e6c812399a4,1
7,2016,64,2016-03-18,3,Texas A&M,Aggies,ef184799-d79b-49d9-a662-991eaf4044cd,14,Green Bay,Phoenix,61a3e5ab-1be3-4694-b83f-edae0953f409,1
8,2016,68,2016-03-15,16,Florida Gulf Coast,Eagles,5326c079-3dc3-468f-a601-b776dd9c1e62,16,Fairleigh Dickinson,Knights,b71d5a1b-2671-4e5a-b94b-06bfb22a27dd,1
9,2016,64,2016-03-17,9,Providence,Friars,7d3b9c72-cbdd-4b64-9aa3-a5bde9b75fdc,8,USC,Trojans,3a000455-de7c-4ca8-880e-abdce7f21da9,1


In [232]:
tourn_games = pd.DataFrame(columns=columns,index=range( tourn_data_2016.shape[0]),dtype='int32')
print(tourn_games.shape)
print(games.shape)
tourn_games[['season','round','game_date','game_result','team','team_seed','team_id',
             'opp_team_seed','opp_team','opp_team_id']] = games[['season','round','game_date','game_result','team','team_seed',
                                                                'team_id','opp_team_seed','opp_team','opp_team_id']]

#tourn_games[['season','round','game_date']] = games[['season','round','game_date']]
tourn_games

(67, 40)
(67, 12)


,season,round,game_date,game_result,team,team_seed,team_id,opp_team,opp_team_seed,opp_team_id,...,opp_allow_off_rebs_avg,opp_def_rebs_avg,opp_allow_def_rebs_avg,opp_ft_att_avg,opp_allow_ft_att_avg,opp_ft_pct,opp_turnover_avg,opp_take_away_avg,opp_win_pct,opp_off_rating
0,2016,64,2016-03-18,1,Virginia Commonwealth,10,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,Oregon State,7,532d3874-b4b3-4c5c-acc6-749a6db26c8f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,64,2016-03-17,1,Utah,3,0d037a5d-827a-44dd-8b70-57603d671d5d,Fresno State,14,dd8db4d8-d984-4cab-b7f6-22c6b8c2c45f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,64,2016-03-18,1,Oregon,1,1da70895-f77f-44ef-b216-d63c02e696eb,Holy Cross,16,31aedd91-a77e-46c1-8bdc-80e9860c159d,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,16,2016-03-24,1,Oregon,1,1da70895-f77f-44ef-b216-d63c02e696eb,Duke,4,faeb1160-5d15-4f26-99fc-c441cf21fc7f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,32,2016-03-20,1,Oregon,1,1da70895-f77f-44ef-b216-d63c02e696eb,Saint Joseph's (PA),8,f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2016,64,2016-03-18,1,Saint Joseph's (PA),8,f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8,Cincinnati,9,a17dfc54-415f-49c3-a2e6-d6fa9db18b0a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2016,32,2016-03-20,1,Texas A&M,3,ef184799-d79b-49d9-a662-991eaf4044cd,Northern Iowa,11,2c93a75d-5449-4ec1-9ca2-1e6c812399a4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2016,64,2016-03-18,1,Texas A&M,3,ef184799-d79b-49d9-a662-991eaf4044cd,Green Bay,14,61a3e5ab-1be3-4694-b83f-edae0953f409,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2016,68,2016-03-15,1,Florida Gulf Coast,16,5326c079-3dc3-468f-a601-b776dd9c1e62,Fairleigh Dickinson,16,b71d5a1b-2671-4e5a-b94b-06bfb22a27dd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2016,64,2016-03-17,1,Providence,9,7d3b9c72-cbdd-4b64-9aa3-a5bde9b75fdc,USC,8,3a000455-de7c-4ca8-880e-abdce7f21da9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [212]:
tourn_games[['opp_team_seed','opp_team','opp_team_id']] = even_losers[['opp_team_seed','opp_team','opp_team_id']]


,season,round,game_date,game_result,team,team_seed,team_id,opp_team,opp_team_seed,opp_team_id,...,opp_allow_off_rebs_avg,opp_def_rebs_avg,opp_allow_def_rebs_avg,opp_ft_att_avg,opp_allow_ft_att_avg,opp_ft_pct,opp_turnover_avg,opp_take_away_avg,opp_win_pct,opp_off_rating
0,2016.0,64.0,2016-03-18,1.0,Virginia Commonwealth,10.0,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,Oregon State,7.0,532d3874-b4b3-4c5c-acc6-749a6db26c8f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016.0,64.0,2016-03-18,1.0,Oregon,1.0,1da70895-f77f-44ef-b216-d63c02e696eb,Holy Cross,16.0,31aedd91-a77e-46c1-8bdc-80e9860c159d,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016.0,32.0,2016-03-20,1.0,Oregon,1.0,1da70895-f77f-44ef-b216-d63c02e696eb,Saint Joseph's (PA),8.0,f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016.0,32.0,2016-03-20,1.0,Texas A&M,3.0,ef184799-d79b-49d9-a662-991eaf4044cd,Northern Iowa,11.0,2c93a75d-5449-4ec1-9ca2-1e6c812399a4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016.0,68.0,2016-03-15,1.0,Florida Gulf Coast,16.0,5326c079-3dc3-468f-a601-b776dd9c1e62,Fairleigh Dickinson,16.0,b71d5a1b-2671-4e5a-b94b-06bfb22a27dd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2016.0,32.0,2016-03-20,1.0,Syracuse,10.0,8cd24de1-2be8-4b77-bcfa-47be0495a5f4,Middle Tennessee,15.0,3d2f0819-8f3b-4ff3-afd6-6227c469a751,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2016.0,64.0,2016-03-18,1.0,Syracuse,10.0,8cd24de1-2be8-4b77-bcfa-47be0495a5f4,Dayton,7.0,632616c5-2dbb-4017-a449-c9dfc303f026,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2016.0,64.0,2016-03-18,1.0,Wisconsin,7.0,c7569eae-5b93-4197-b204-6f3a62146b25,Pittsburgh,10.0,24051034-96bb-4f78-a3a6-312f3258780f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2016.0,64.0,2016-03-17,1.0,Connecticut,9.0,efba5350-79bc-47aa-a189-db177b95b491,Colorado,8.0,9fccbf28-2858-4263-821c-fdefb3c7efa3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2016.0,8.0,2016-03-26,1.0,Oklahoma,2.0,b2fda957-e15c-4fb2-8a13-6e58496f561e,Oregon,1.0,1da70895-f77f-44ef-b216-d63c02e696eb,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
odd_games =tourn_data_2016.iloc[1::2].reset_index(drop=True)
odd_games

,season,round,game_date,win_seed,win_market,win_name,win_team_id,win_kaggle_team_id,win_pts,lose_seed,lose_market,lose_name,lose_team_id,lose_school_ncaa,lose_kaggle_team_id,lose_pts
0,2016,64,2016-03-17,3,Utah,Utes,0d037a5d-827a-44dd-8b70-57603d671d5d,1428,80,14,Fresno State,Bulldogs,dd8db4d8-d984-4cab-b7f6-22c6b8c2c45f,Fresno St.,1201,69
1,2016,16,2016-03-24,1,Oregon,Ducks,1da70895-f77f-44ef-b216-d63c02e696eb,1332,82,4,Duke,Blue Devils,faeb1160-5d15-4f26-99fc-c441cf21fc7f,Duke,1181,68
2,2016,64,2016-03-18,8,Saint Joseph's (PA),Hawks,f9e4261e-d11d-46c4-bd33-c7bbc94ef0e8,1386,78,9,Cincinnati,Bearcats,a17dfc54-415f-49c3-a2e6-d6fa9db18b0a,Cincinnati,1153,76
3,2016,64,2016-03-18,3,Texas A&M,Aggies,ef184799-d79b-49d9-a662-991eaf4044cd,1401,92,14,Green Bay,Phoenix,61a3e5ab-1be3-4694-b83f-edae0953f409,Green Bay,1453,65
4,2016,64,2016-03-17,9,Providence,Friars,7d3b9c72-cbdd-4b64-9aa3-a5bde9b75fdc,1344,70,8,USC,Trojans,3a000455-de7c-4ca8-880e-abdce7f21da9,Southern California,1425,69
5,2016,16,2016-03-25,10,Syracuse,Orange,8cd24de1-2be8-4b77-bcfa-47be0495a5f4,1393,63,11,Gonzaga,Bulldogs,2f4d21f8-6d5f-48a5-abca-52a30583871a,Gonzaga,1211,60
6,2016,8,2016-03-27,10,Syracuse,Orange,8cd24de1-2be8-4b77-bcfa-47be0495a5f4,1393,68,1,Virginia,Cavaliers,56913910-87f7-4ad7-ae3b-5cd9fb218fd9,Virginia,1438,62
7,2016,32,2016-03-20,7,Wisconsin,Badgers,c7569eae-5b93-4197-b204-6f3a62146b25,1458,66,2,Xavier,Musketeers,0d8a328f-20ce-410b-bed1-15fc01308aaf,Xavier,1462,63
8,2016,16,2016-03-24,2,Oklahoma,Sooners,b2fda957-e15c-4fb2-8a13-6e58496f561e,1328,77,3,Texas A&M,Aggies,ef184799-d79b-49d9-a662-991eaf4044cd,Texas A&M,1401,63
9,2016,32,2016-03-20,2,Oklahoma,Sooners,b2fda957-e15c-4fb2-8a13-6e58496f561e,1328,85,10,Virginia Commonwealth,Rams,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,VCU,1433,81


In [135]:
tourn_games.season =pd.concat([tourn_games.season,odd_games.season],ignore_index=False)
tourn_games

ValueError: cannot reindex from a duplicate axis

In [131]:
tourn_games.shape

(34, 38)